# FDMT efficiency
The current FDMT seems to be a bit inefficient - i.e. we do the FDMT of a pulse and the peak in the FDMT should be the sum of the pulse - and it's not. Worrying.

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import numpy as np
from craft import fdmt # you'll need to have ../python in  you PYTHONPATH
import simfrb # in same directory as fdmt
from pylab import *

ModuleNotFoundError: No module named 'simfrb'

In [ ]:
nf = 256 # number of channels - non-power-of-2 - naughty naughty!
fmax = 1440. #  Freuency of the top of the band in MHz
df = 1.0 # Channel bandwidth in MHz
fmin = fmax - nf*df # Frequency of the bottom of the band in MHz
nd = 512*2 # Number of DM trials to do
nt = 256 # Number of samples per block
tsamp = 1.0 # milliseconds

In [ ]:
def mysimfrb(fmin, df, nf, tsamp, idt, nt, offset, width=0):
    assert df > 0
    assert nf > 0
    assert fmin > 0
    d = np.zeros((nf, nt))
    fmax = fmin + nf*df
    fmin /= 1e3
    fmax /= 1e3
    df /= 1e3
    dm1 = 4.15*(fmin**-2 - fmax**-2)
    dm = idt*tsamp/dm1
    
    assert dm1> 0
    
    for c in range(nf):
        f2 = fmin + c*df       
        offset_ms = -4.15*dm*(fmax**-2 - f2**-2) + offset
        offset_samp = int(np.round(offset_ms/tsamp)) 
        #print c, f2, fmax, offset_ms, offset_samp
        d[c, offset_samp] = 1
    return d

In [ ]:
def checkhit(thefdmt, dm, offset=0):
    # I haven't had time to tidy this up - the untis are funny and the FRB is updside down and transposed
    # With respect to what the FDMT wants to see - just trust me OK.
    # Caution: The DM units here are in pc/cm3 and the DM units the FDMT uses are in samples. at about 1 GHz and 1 ms these are almost identical.
    #frb = simfrb.mkfrb(fmax/1e3, -df/1e3, nf, tsamp=tsamp, dm=dm, ntimes=nt, offset=10).T
    #frb = np.flipud(frb)
    frb = mysimfrb(fmin, df, nf, tsamp, dm, nt, offset=offset)
    fout = thefdmt(frb)
    hitting_efficiency = fout.max()/frb.sum() # should be close to 1. Ideally 1.
    maxidx = fout.argmax()
    idtmax, tmax = np.unravel_index(maxidx, fout.shape)
    dm_of_idt = idtmax*tsamp/(4.15*((fmin/1e3)**-2 - (fmax/1e3)**-2)) # convert idt to pc/cm3
    #print 'Hitting efficiency', hitting_efficiency, 'Maximum at t=', tmax, 'idt=', idtmax, '=', dm_of_idt, 'pc/cm3'
    
    return (hitting_efficiency, idtmax, tmax)

In [ ]:
thefdmt = fdmt.Fdmt( fmin, df, nf, nd, nt) # make FDMT
dmrange = np.arange(0, 10, 10)
hitdata = []
for dm in dmrange:
    hitdata.append(checkhit(thefdmt, dm, offset=10))
    
hitdata = np.array(hitdata)    

In [ ]:
idt = 200



frb1 = mysimfrb(fmin, df, nf, tsamp, idt, nt, offset=0)
frb2 = mysimfrb(fmin, df, nf, tsamp, idt, nt, offset=0.5)
frbfdmt = np.zeros((nf, nt))
frbfdmt = thefdmt.add_frb_track(idt,frbfdmt)


fig, ax = subplots(1,4, sharex=True, sharey=True)
ax[0].imshow(frb1, aspect='auto', origin='lower', interpolation='nearest')
ax[0].set_title(f'mysimfrb offset=0 idt={idt}')

ax[1].imshow(frb2, aspect='auto', origin='lower', interpolation='nearest')
ax[1].set_title(f'mysimfrb offset=0.5 idt={idt}')

ax[2].imshow(frbfdmt, aspect='auto', origin='lower', interpolation='nearest')
ax[2].set_title(f'FDMT idt={idt}')

ax[3].imshow(frbfdmt+frb2, aspect='auto', origin='lower', interpolation='nearest')
ax[3].set_title(f'FDMT+frb0.5 idt={idt}')




xlabel('Sample')
ylabel('channel')

In [ ]:
hitdata.shape

In [ ]:
fig, ax = subplots(1,3, sharex=True)
ax[0].plot(dmrange, hitdata[:, 0], 'o')
ax[0].set_ylabel('Hitting efficiency ')
ax[1].plot(dmrange, hitdata[:, 1], 'o')
ax[1].set_ylabel('IDT max')
ax[2].plot(dmrange, hitdata[:, 2] - hitdata[:, 1], 'o')
ax[2].set_ylabel('tmax')

In [ ]:
dm = 7
frb = mysimfrb(fmin, df, nf, tsamp, dm, nt, offset=10)
fout = thefdmt(frb)

In [ ]:
%timeit thefdmt(frb)

In [ ]:
figure()
imshow(fout, aspect='auto', origin='lower')

In [ ]:
figure()
plot(fout.max(axis=1)/frb.sum())
xlabel('IDT')
ylabel('Sum as a function of IDT')

In [ ]:
frb.sum()

In [ ]:
number_of_sums = thefdmt(np.ones((nf,nt)))

In [ ]:
figure()
imshow(number_of_sums, aspect='auto', origin='lower')
xlabel('Sample')
ylabel('IDT')

In [ ]:
figure()
plot(number_of_sums[:,nt-1])
xlabel('IDT')
ylabel('Number of cells added')

In [ ]:
frblong = mysimfrb(fmin, df, nf, tsamp, 900, 1024, offset=0)


In [ ]:
figure()
imshow(frblong, aspect='auto', origin='lower')

In [ ]:
frblong.shape
def overlapandsum(fdmt_output, sums, nt):
    ndm, ntall = sums.shape
    for idm in range(ndm):
        for t in range(ntall):            
            if t + nt < ntall:
                sums[idm, t] = fdmt_output[idm, t] + sums[idm, t+nt]
            else:
                sums[idm,t] = fdmt_output[idm,t]
   
    return sums

In [ ]:
nt

In [ ]:
fig, axs = subplots(3,4)
fig.set_size_inches([10,5])
sums = np.zeros((nd, nd+nt))
#frblong[:] = 1
for iax in range(4):
    frbblk = frblong[:, iax*256:(iax+1)*256]
    dout = thefdmt(frbblk)  
    overlapandsum(dout, sums, nt)
    axs[0,iax].set_title(f'Block {iax}')
    axs[0,iax].imshow(frbblk, aspect='auto', origin='lower')    
    axs[1,iax].imshow(dout, aspect='auto', origin='lower')
    axs[2,iax].imshow(sums.copy(), aspect='auto', origin='lower')
    
    

axs[0,0].set_ylabel('channel')
axs[1,0].set_ylabel('iDM')
axs[1,0].set_xlabel('sample')

In [ ]:
thefdmt_long = fdmt.Fdmt( fmin, df, nf, nd, nt*4) # make FDMT

longout = thefdmt_long(frblong)
fig,ax = subplots(1,3, sharex=True,sharey=True)
ax[0].imshow(longout[:,3*256:4*256], aspect='auto', origin='lower')
ax[1].imshow(sums[:,0:256], aspect='auto', origin='lower')
ax[2].imshow(sums[:,0:256]-longout[:,3*256:4*256], aspect='auto', origin='lower')


In [ ]:
%timeit thefdmt(frbblk)  

In [ ]:
from numba import njit

#@njit
def boxcar_and_threshold(d, nt, boxcar_history, threshold):
    candidates = [] # candidate list
    ndm, _ = d.shape
    ndm, nbox = boxcar_history.shape
    
    for idm in range(ndm): # loop over DM
        for t in range(nt): # loop over time
            bcsum = np.float32(0) # initialise running sum over up to nbox previous times
            best_cand = None
            best_sn = None
            for ibox in range(nbox): # for each value in the past
                if t >= ibox: # if the previous value is in the current block
                    inv = d[idm, t - ibox]
                else: # we need to go back into the boxcar history
                    inv = boxcar_history[idm, -ibox]
                    
                bcsum += inv # add the previous value
                sn = bcsum / np.sqrt(ibox+1) # scale by sqrt width
                # before outputting a candidate that was the best over the boxcars & times
                if sn >= threshold: # output candidate if it exceeds the threshold
                    if best_sn is not None and sn > best_sn:
                        best_cand = (idm,t,ibox,sn)
                        best_sn = sn
                else:
                    if best_cand is not None:
                        # We've gone back below the threshold
                        candidates.append(best_cand)
                        best_cand = None
                        best_sn = None
                    
    boxcar_history[:,:] = d[:,-nbox:]
                

    return candidates

In [ ]:

boxcar_history = np.zeros((nd,32), dtype=np.float32)
boxcar_and_threshold(sums.astype(float32)/sums.max()*10+np.random.randn(sums.size).reshape(sums.shape)*1, nt,boxcar_history, 10)

In [ ]:
%timeit boxcar_and_threshold(sums.astype(float32), nt,boxcar_history, 20)

In [ ]:
sums.max()